<a href="https://colab.research.google.com/github/SantiagoLizarazo/PROGCOM-B/blob/main/HardCore_Quest_Three%20-%20SEGUNDO%20CORTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pygame
import sys
import random

#Configuración Visual
ANCHO = 1920
ALTO = 1080
NEGRO = (0, 0, 0)
BLANCO = (255, 255, 255)
GRIS_MARCA = (150, 150, 150) # Gris para las marcas
GRIS_LINEA = (200, 200, 200) # Líneas un poco más claras

# Dimensiones del Tablero
TAMANO_TABLERO = 750
TAMANO_CELDA = TAMANO_TABLERO // 3
GROSOR_LINEA = 8

OFFSET_X = (ANCHO - TAMANO_TABLERO) // 2
OFFSET_Y = (ALTO - TAMANO_TABLERO) // 2 - 50

# Estados de Juego
ESTADO_SELECCION_SIMBOLO = 0
ESTADO_JUEGO = 1
ESTADO_FINALIZADO_RONDA = 2

# Inicializar Pygame
pygame.init()
pantalla = pygame.display.set_mode((ANCHO, ALTO))
pygame.display.set_caption("TIC-TAC-TOE: Estilo Minimalista (v4)")
reloj = pygame.time.Clock()

# Fuente
FUENTE_TITULO = pygame.font.SysFont('Arial', 80, bold=True)
FUENTE_MARCADOR = pygame.font.SysFont('Arial', 50)
FUENTE_INSTRUCCION = pygame.font.SysFont('Arial', 40)
FUENTE_BOTON_SIMBOLO = pygame.font.SysFont('Arial', 40)

juego_estado = ESTADO_SELECCION_SIMBOLO
tablero = [' '] * 9
jugador_humano = ''
jugador_maquina = ''
turno = ''

marcador = {'X': 0, 'O': 0, 'Empates': 0}

def reiniciar_tablero():
    """Limpia el tablero y pasa al siguiente turno."""
    global tablero, turno, juego_estado
    tablero = [' '] * 9
    turno = jugador_humano
    juego_estado = ESTADO_JUEGO

def verificar_ganador(tablero, jugador):
    """Verifica si el jugador actual ha ganado."""
    combinaciones = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8), # Filas
        (0, 3, 6), (1, 4, 7), (2, 5, 8), # Columnas
        (0, 4, 8), (2, 4, 6)             # Diagonales
    ]
    for a, b, c in combinaciones:
        if tablero[a] == tablero[b] == tablero[c] == jugador:
            return True
    return False

def obtener_casillas_vacias(tablero):
    """Devuelve una lista de los índices de las casillas vacías."""
    return [i for i, marca in enumerate(tablero) if marca == ' ']

def movimiento_ia(tablero):
    """Lógica de la máquina: ganar -> bloquear -> centro -> esquina -> lado."""
    casillas_vacias = obtener_casillas_vacias(tablero)
    if not casillas_vacias: return None

    # 1. Ganar
    for i in casillas_vacias:
        temp_tablero = list(tablero)
        temp_tablero[i] = jugador_maquina
        if verificar_ganador(temp_tablero, jugador_maquina): return i

    # 2. Bloquear
    for i in casillas_vacias:
        temp_tablero = list(tablero)
        temp_tablero[i] = jugador_humano
        if verificar_ganador(temp_tablero, jugador_humano): return i

    # 3. Estrategia (Centro > Esquinas > Lados)
    if 4 in casillas_vacias: return 4
    esquinas = [i for i in [0, 2, 6, 8] if i in casillas_vacias]
    if esquinas: return random.choice(esquinas)

    return random.choice(casillas_vacias)

def dibujar_marcador():
    """Dibuja el marcador de victorias y empates al pie del tablero."""

    # Marcador Jugador (X)
    texto_x = FUENTE_MARCADOR.render(f"JUGADOR (X): {marcador['X']}", True, BLANCO if jugador_humano == 'X' else GRIS_MARCA)
    rect_x = texto_x.get_rect(center=(ANCHO // 2 - 300, ALTO - 150))
    pantalla.blit(texto_x, rect_x)

    # Marcador Empates
    texto_empate = FUENTE_MARCADOR.render(f"EMPATES: {marcador['Empates']}", True, BLANCO)
    rect_empate = texto_empate.get_rect(center=(ANCHO // 2, ALTO - 150))
    pantalla.blit(texto_empate, rect_empate)

    # Marcador Máquina (O)
    texto_o = FUENTE_MARCADOR.render(f"MÁQUINA (O): {marcador['O']}", True, BLANCO if jugador_humano == 'O' else GRIS_MARCA)
    rect_o = texto_o.get_rect(center=(ANCHO // 2 + 300, ALTO - 150))
    pantalla.blit(texto_o, rect_o)

def dibujar_tablero():
    """Dibuja las líneas del tablero y las marcas X/O."""

    for i in range(1, 3):
        # Vertical
        inicio_v = (OFFSET_X + i * TAMANO_CELDA, OFFSET_Y)
        fin_v = (OFFSET_X + i * TAMANO_CELDA, OFFSET_Y + TAMANO_TABLERO)
        pygame.draw.line(pantalla, GRIS_LINEA, inicio_v, fin_v, GROSOR_LINEA)
        # Horizontal
        inicio_h = (OFFSET_X, OFFSET_Y + i * TAMANO_CELDA)
        fin_h = (OFFSET_X + TAMANO_TABLERO, OFFSET_Y + i * TAMANO_CELDA)
        pygame.draw.line(pantalla, GRIS_LINEA, inicio_h, fin_h, GROSOR_LINEA)

    for i in range(9):
        fila = i // 3
        columna = i % 3

        centro_x = OFFSET_X + columna * TAMANO_CELDA + TAMANO_CELDA // 2
        centro_y = OFFSET_Y + fila * TAMANO_CELDA + TAMANO_CELDA // 2

        color_marca = BLANCO if tablero[i] == jugador_humano else GRIS_MARCA

        if tablero[i] == 'X':
            # Dibujar X
            pygame.draw.line(pantalla, color_marca, (centro_x - 80, centro_y - 80), (centro_x + 80, centro_y + 80), 12)
            pygame.draw.line(pantalla, color_marca, (centro_x + 80, centro_y - 80), (centro_x - 80, centro_y + 80), 12)
        elif tablero[i] == 'O':
            # Dibujar O
            pygame.draw.circle(pantalla, color_marca, (centro_x, centro_y), 80, 12)

def dibujar_boton(texto, x, y, ancho, alto, color_fondo, color_texto, fuente):
    """Dibuja un botón interactivo y devuelve su rectángulo, usando una fuente específica."""
    rect = pygame.Rect(x, y, ancho, alto)
    pygame.draw.rect(pantalla, color_fondo, rect, 0, 10)

    superficie_texto = fuente.render(texto, True, color_texto)
    rect_texto = superficie_texto.get_rect(center=rect.center)
    pantalla.blit(superficie_texto, rect_texto)
    return rect

def dibujar_seleccion_simbolo():
    """Estado 0: Permite al jugador elegir X u O."""
    pantalla.fill(NEGRO)

    texto_titulo = FUENTE_TITULO.render("Elige tu Símbolo", True, BLANCO)
    pantalla.blit(texto_titulo, (ANCHO // 2 - texto_titulo.get_width() // 2, 200))

    texto_instruccion = FUENTE_INSTRUCCION.render("Jugador 1 (Humano) | Jugador 2 (Máquina)", True, GRIS_LINEA)
    pantalla.blit(texto_instruccion, (ANCHO // 2 - texto_instruccion.get_width() // 2, 300))

    rect_x = dibujar_boton("JUGAR COMO X", ANCHO // 2 - 350, ALTO // 2, 300, 80, GRIS_LINEA, NEGRO, FUENTE_BOTON_SIMBOLO)

    rect_o = dibujar_boton("JUGAR COMO O", ANCHO // 2 + 50, ALTO // 2, 300, 80, GRIS_LINEA, NEGRO, FUENTE_BOTON_SIMBOLO)

    return rect_x, rect_o

while True:
    for evento in pygame.event.get():
        if evento.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        if evento.type == pygame.MOUSEBUTTONDOWN:
            pos_x, pos_y = evento.pos

            if juego_estado == ESTADO_SELECCION_SIMBOLO:
                rect_x, rect_o = dibujar_seleccion_simbolo()
                if rect_x.collidepoint(pos_x, pos_y):
                    jugador_humano = 'X'
                    jugador_maquina = 'O'
                    reiniciar_tablero()
                elif rect_o.collidepoint(pos_x, pos_y):
                    jugador_humano = 'O'
                    jugador_maquina = 'X'
                    reiniciar_tablero()

            elif juego_estado == ESTADO_JUEGO and turno == jugador_humano:


                if (pos_x >= OFFSET_X and pos_x <= OFFSET_X + TAMANO_TABLERO and
                    pos_y >= OFFSET_Y and pos_y <= OFFSET_Y + TAMANO_TABLERO):


                    columna = (pos_x - OFFSET_X) // TAMANO_CELDA
                    fila = (pos_y - OFFSET_Y) // TAMANO_CELDA
                    indice = fila * 3 + columna

                    if tablero[indice] == ' ':
                        tablero[indice] = jugador_humano


                        if verificar_ganador(tablero, jugador_humano):
                            marcador[jugador_humano] += 1
                            juego_estado = ESTADO_FINALIZADO_RONDA
                        elif not obtener_casillas_vacias(tablero):
                            marcador['Empates'] += 1
                            juego_estado = ESTADO_FINALIZADO_RONDA
                        else:
                            turno = jugador_maquina

            elif juego_estado == ESTADO_FINALIZADO_RONDA:
                # Cualquier click reanuda/reinicia la ronda
                reiniciar_tablero()

        if evento.type == pygame.KEYDOWN:
            if evento.key == pygame.K_r: # R para reiniciar (puntuación)
                marcador = {'X': 0, 'O': 0, 'Empates': 0}
                juego_estado = ESTADO_SELECCION_SIMBOLO
            if evento.key == pygame.K_q: # Q para cerrar
                pygame.quit()
                sys.exit()

    if juego_estado == ESTADO_JUEGO and turno == jugador_maquina:
        pygame.time.wait(500) # Pausa para simular el "pensamiento"

        indice_ia = movimiento_ia(tablero)

        if indice_ia is not None:
            tablero[indice_ia] = jugador_maquina

            # Chequear fin de ronda
            if verificar_ganador(tablero, jugador_maquina):
                marcador[jugador_maquina] += 1
                juego_estado = ESTADO_FINALIZADO_RONDA
            elif not obtener_casillas_vacias(tablero):
                # ¡Empate detectado!
                marcador['Empates'] += 1
                juego_estado = ESTADO_FINALIZADO_RONDA
            else:
                turno = jugador_humano
        elif not obtener_casillas_vacias(tablero) and juego_estado == ESTADO_JUEGO:
            # Caso de empate si la IA no encuentra jugada pero la lista está vacía
            marcador['Empates'] += 1
            juego_estado = ESTADO_FINALIZADO_RONDA

    pantalla.fill(NEGRO) # Fondo negro

    if juego_estado == ESTADO_SELECCION_SIMBOLO:
        dibujar_seleccion_simbolo()

    elif juego_estado in (ESTADO_JUEGO, ESTADO_FINALIZADO_RONDA):
        dibujar_tablero()
        dibujar_marcador()

        # Mensaje de estado
        if juego_estado == ESTADO_JUEGO:
            texto_turno = FUENTE_INSTRUCCION.render(f"Tu símbolo: {jugador_humano}. Turno de: {turno}", True, GRIS_LINEA)
            pantalla.blit(texto_turno, (10, 10))

        # Superposición de mensaje final
        if juego_estado == ESTADO_FINALIZADO_RONDA:

            # Determinar el mensaje
            if verificar_ganador(tablero, jugador_humano):
                 mensaje = "¡Ganaste la ronda!"
            elif verificar_ganador(tablero, jugador_maquina):
                 mensaje = "¡La máquina ganó la ronda!"
            else:
                 # Mensaje de empate
                 mensaje = "¡Empatamos! ¿Reiniciar partida?"

            s = pygame.Surface((ANCHO, ALTO), pygame.SRCALPHA)
            s.fill((0, 0, 0, 200))
            pantalla.blit(s, (0, 0))

            texto_final = FUENTE_TITULO.render(mensaje, True, BLANCO)
            texto_instruccion = FUENTE_INSTRUCCION.render("Clic para empezar la siguiente ronda", True, GRIS_LINEA)

            pantalla.blit(texto_final, (ANCHO // 2 - texto_final.get_width() // 2, ALTO // 2 - 100))
            pantalla.blit(texto_instruccion, (ANCHO // 2 - texto_instruccion.get_width() // 2, ALTO // 2 + 50))

            # Instrucciones de menú
            texto_menu_inst = FUENTE_INSTRUCCION.render("Presiona R para Reiniciar Puntuación | Presiona Q para Salir", True, GRIS_LINEA)
            pantalla.blit(texto_menu_inst, (ANCHO // 2 - texto_menu_inst.get_width() // 2, ALTO - 50))

    pygame.display.flip()
    reloj.tick(60)